# Dessiner récursivement

## `ipycanvas`

Nous utilisons le module `ipycanvas` - [doc](https://ipycanvas.readthedocs.io/en/latest/index.html) - permettant de dessiner dans un «canvas» intégré au notebook.

In [ ]:
from ipycanvas import Canvas

Il suffit alors de le «construire» (en lui fournissant éventuellement sa largeur *width* et sa hauteur *height*):

In [ ]:
can = Canvas(width=400, height=400)
can

**Note**: pour garder en vue la zone affichée, ouvrir une autre vue du même notebook (clic droit sur le nom du notebook et choisir «New View for Notebook»).

### Couleur de remplissage

On définit la couleur de remplissage `.fill_style = <couleur>` et on remplit un rectangle `.fill_rect(x, y, w, h)`.

Nous encadrons ces commandes avec `.save()` et `.restore()` afin de remettre la couleur de remplissage par défaut à sa valeur initiale.

In [ ]:
can.save()
can.fill_style = "cornsilk" # voir https://developer.mozilla.org/fr/docs/Web/CSS/Type_color
can.fill_rect(0, 0, can.width, can.height)
can.restore()# remet la couleur de remplissage par défaut

### Dessiner sur le canvas

Le canvas est rapporté un à repère comme suit:
![](https://ipycanvas.readthedocs.io/en/latest/_images/grid.png)

#### Des rectangles pour commencer

ExerciceUn rectangle est défini par les coordonnées de son point supérieur gauche (*topleft*), sa largeur (*width*) et sa hauteur (*height*).

L'opération `.fill_rect(x, y, width, height)` remplit le rectangle donné avec la couleur de remplissage `.fill_rect`.

L'opération `.stroke_rect(<rect>)` dessine son **contour** avec la couleur `.stroke_style`.

Voici un exemple simple:

In [ ]:
y = can.height / 2
for x in range(10, can.width-10, 20):
    can.fill_rect(x, y, 10, 10)
    can.save()
    can.stroke_style = "red"
    can.stroke_rect(x, y, 10, 10)
    can.restore()

`.clear()` pour effacer le canvas.

In [ ]:
can.clear()

ExerciceMais on souhaite conserver une couleur de fond... alors définissons une fonction pour cela:

In [ ]:
def clear(couleur="cornsilk"):
    can.clear()
    can.save()
    can.fill_style = couleur # voir https://developer.mozilla.org/fr/docs/Web/CSS/Type_color
    can.fill_rect(0, 0, can.width, can.height)
    can.restore() # remet la couleur de remplissage par défaut

In [ ]:
clear()

#### Exercice 1

1. En reprenant l'exemple précédent, afficher une colonne de petits rectangles noirs (plutôt qu'une ligne)

In [ ]:
# votre solution

**Solution**

In [ ]:
clear()
x = can.width / 2
for y in range(10, can.height-10, 20):
    can.fill_rect(x, y, 10, 10)
    can.save()
    can.stroke_style = "red"
    can.stroke_rect(x, y, 10, 10)
    can.restore()

2. Remplir tout le canvas avec ces petits rectangle (boucles imbriqués)

In [ ]:
# votre solution

**Solution**

In [ ]:
clear()
for x in range(10, can.height-10, 20):
    for y in range(10, can.width-10, 20):
        can.fill_rect(x, y, 10, 10)
        can.save()
        can.stroke_style = "red"
        can.stroke_rect(x, y, 10, 10)
        can.restore()

### Tracer une ligne polygonale

On déclare qu'on va définir une telle ligne avec `.begin_path()`(*path*: chemin), puis on place son point de départ `.move_to(x, y)` et on définit des lignes avec `.line_to(x, y)` (la position courante du tracé est mise à jour).

Une fois la ligne polygonale définie, on la trace avec `.stroke()`.

Pour bien comprendre, nous allons ralentir le tracé avec la fonction `sleep(<secondes>)` du module `time`. Cela suspend python le temps indiqué.

In [ ]:
from time import sleep

Voici un exemple basique:

In [ ]:
clear()
x0, x1, x2, x3 = 10,  50, 100, 20
y0, y1, y2, y3 = 20, 100,  50, 10
can.begin_path()
can.move_to(x0,y0)
# on peut séparer plusieurs instructions sur une même ligne
# avec «;»
can.line_to(x1,y1); can.stroke(); sleep(.5) 
can.line_to(x2,y2); can.stroke(); sleep(.5)
can.line_to(x3,y3); can.stroke(); sleep(.5)

En voici un autre:

In [ ]:
clear()
can.begin_path()
x, y = 10, can.height / 2
can.move_to(x,y)
for i in range(100):
    x += 10
    if x > can.width - 10:
        break
    y += 10 if i % 2 == 0 else -10
    can.line_to(x,y)
can.stroke() # il est plus efficace de dessiner le chemin en une fois

Voilà pour le principe. À présent, plutôt que de fournir (ou de calculer) chaque point au fur et à mesure, on peut les fournir dans une liste à une fonction `poly` qui se chargera de dessiner la ligne polygonale correspondante.

#### Exercice 2

Voici un point de départ à compléter:

In [ ]:
# pts est une liste de tuples de la forme (x, y)
def poly(pts, fermer=True):
    if len(pts) < 2: return
    depart, *autres = pts
    can.___()
    can.move_to(*depart) # equivalent à `x, y = depart; can.move_to(x, y)`
    for pt in autres:
        can.line_to(___)
    if fermer:
        can.close_path()
    can.___()

Pour la tester

In [ ]:
clear()
pts = [
    (1/4 * can.width, 2/3 * can.height),
    (3/4 * can.width, 2/3 * can.height),
    (1/2 * can.width, 1/4 * can.height)
]
poly(pts) # essayer poly(pts, fermer=False) pour voir la différence...

**Solution**

In [ ]:
def poly(pts, fermer=True):
    if len(pts) < 2: return
    depart, *autres = pts
    can.begin_path()
    can.move_to(*depart) # equivalent à `x, y = depart; can.move_to(x, y)`
    for pt in autres:
        can.line_to(*pt)
    if fermer:
        can.close_path()
    can.stroke()

***

## Triangle de Sierpinsky

On part d'un triangle (souvent équilatéral... mais pas obligatoirement), et on en construit d'autres selon le schéma:

![](https://mathworld.wolfram.com/images/eps-gif/SierpinskiGraph_800.gif)

Pour dessiner une figure analogue, nous aurons clairement besoin de calculer le point milieu de deux points donnés.

#### Exercice 3

compléter le code qui suit

In [ ]:
# pt1 et pt2 sont supposés être des tuples de la forme (x, y)
def milieu(pt1, pt2):
    x1, y1 = pt1
    ___
    return ____, ____

In [ ]:
assert milieu((100, 50), (200, 150)) == (150, 100)

**Solution**

In [ ]:
def milieu(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    return (x1 + x2) / 2, (y1 + y2) / 2

On peut alors utiliser cette fonction pour calculer **les** milieux des segments d'une ligne polygonale fournie sous la forme d'une liste de points:

In [ ]:
def milieux(pts, fermer=True):
    if len(pts) < 2: return []
    ms = []
    A, *autres = pts
    for B in autres:
        ptm = milieu(A, B)
        ms.append(ptm)
        A = B
    if fermer:
        ms.append(milieu(pts[0],A))
    return ms

In [ ]:
clear()
poly(pts)
poly(milieux(pts))

#### Exercice 4

En exploitant la récursivité multiple, vous devriez réussir à dessiner votre propre triangle de Sierpinsky:

Compléter la fonction qui suit pour y parvenir:

In [ ]:
def sierpinsky(pts, n):
    if n == -1: return
    poly(pts)
    ms = milieux(pts)
    sierpinsky([pts[0],ms[0],ms[2]], n-1)
    sierpinsky(___, n-1)
    sierpinsky(___, n-1)

puis tester:

In [ ]:
clear()
tri = [
    (0, can.height),
    (can.width, can.height),
    (can.width/2, 0)
]
sierpinsky(tri, 5)

**Solution**

In [ ]:
def sierpinsky(pts, n):
    if n == -1: return
    poly(pts)
    ms = milieux(pts)
    sierpinsky([pts[0],ms[0],ms[2]], n-1)
    sierpinsky([pts[1],ms[0],ms[1]], n-1)
    sierpinsky([pts[2],ms[2],ms[1]], n-1)

***

Si on prend `n=7` dans le code précédent, le dessin ne vas pas au bout. En effet, chaque appel de commande de dessin comme `.stroke` envoie un «message» au canvas et le nombre limite de tels messages est de 1000. Or:

In [ ]:
3**6, 3**7

Mais, on peut tout de même effectuer le tracer à condition d'envoyer toutes les commandes en une fois.

Pour cela, on utilise la syntaxe:
```python
with hold_canvas(can):
    ## dessins
```

Il est nécessaire d'importer `hold_canvas`:
```python
from ipycanvas import hold_canvas
```

Ici, cela donne:

In [ ]:
from ipycanvas import hold_canvas

In [ ]:
clear()
with hold_canvas(can):
    sierpinsky(tri, 7)

#### Un peu de couleur (complément)

On pourrait mettre un peu de couleur... en adaptant `poly` et `sierpinsky`:

In [ ]:
palette = ["black", "red", "green", "blue", "white"]

def poly_col(pts, color="white"):
    if len(pts) < 2: return
    depart, *autres = pts
    can.begin_path()
    can.move_to(*depart)
    for pt in autres:
        can.line_to(*pt)
    can.close_path()
    can.save()
    can.fill_style = color
    can.fill()
    can.restore()

def sierpinsky_col(pts, n):
    if n == -1: return
    poly_col(
        pts, 
        palette[n%len(palette)]
    )
    ms = milieux(pts)
    sierpinsky_col([pts[0],ms[0],ms[2]], n-1)
    sierpinsky_col([pts[1],ms[0],ms[1]], n-1)
    sierpinsky_col([pts[2],ms[2],ms[1]], n-1)

In [ ]:
clear()
# with hold_canvas(can):
sierpinsky_col(tri, 6)

## Tapis de Sierpinsky

On commence par dessiner un rectangle.

In [ ]:
rect_base = (
    .1 * can.width, .1 * can.height, # x, y
    .8 * can.width, .8 * can.height # w, h
)
clear()
can.fill_rect(*rect_base)

Puis, on le divise en 9 rectangles de même taille en divisant la
largeur et la hauteur du rectangle initial par 3.

On retire alors le rectangle du milieu (en le remplissant de blanc par exemple)

In [ ]:
x, y, w, h = rect_base
rect_centre = (
    x + 1/3 * w, y + 1/3 * h,
    w/3, h/3
)
can.save()
can.fill_style = "white"
can.fill_rect(*rect_centre)
can.restore()

L'idée est alors de recommencer avec les 8 sous-rectangles restants...

![](https://s3.amazonaws.com/illustrativemathematics/images/000/002/320/max/2_f84d6f2ea4f799167285abcb4002e75e.jpg?1369368754)

Il y a intérêt à s'organiser...

In [ ]:
def trouer_rectangle(rect):
    x, y, w, h = rect
    can.save()
    can.fill_style = "white"
    can.fill_rect(x+w/3, y+h/3, w/3, h/3)
    can.restore()
    # construire la liste des 8 sous-rectangles
    sous_rects = []
    for i in range(3):
        for j in range(3):
            if not (i == 1 and j == 1):
                sous_rects.append((
                    x + i * w/3, y + j * h/3,
                    w/3, h/3
            ))
    return sous_rects

on peut alors procéder comme suit:

In [ ]:
clear()
can.fill_rect(*rect_base)
sous_rs = trouer_rectangle(rect_base)
for r in sous_rs:
    trouer_rectangle(r)

Mais on pourrait recommencer avec les 8 sous-sous-rectangles des sous rectangles, et puis faire cela à nouveau...

#### Exercice 5

En réutilisant `trouer_rectangle`, définir `tapis(rect, n)` qui dessine un tapis de Sierpinsky de «niveau» `n` dans le rectangle `rect` fourni.

Pour `n=0`, On affiche le rectangle de base, pour `n=1`, il est troué, pour `n=2`, les sous-rectangles le sont à leur tour et ainsi de suite.

In [ ]:
def tapis(rect, n):
    # à vous de jouer

In [ ]:
clear()
tapis(rect_base, n=3)

Cela échoue pour `n=4` car $8^4=(2^3)^4=2^{12} > 1000$

In [ ]:
clear()
with hold_canvas(can):
    tapis(rect_base, n=4)

**Solution**

In [ ]:
def tapis(rect, n):
    if n == 0:
        can.save()
        can.fill_style = "black"
        can.fill_rect(*rect)
        can.restore()
        return
    rs = trouer_rectangle(rect)
    for r in rs:
        tapis(r, n-1)

***